In [4]:
from reweighing import weighting_educ, weighting_maritalstatus
import os 

In [12]:
# applying weights
import csv
gender = []
educ = []
status = []
with open(os.path.join("..","Processed_data",'adult.csv'), newline='') as data:
    spamreader = csv.reader(data, delimiter=',')
    for row in spamreader:
        #[1:-1] because the words look like this: "'word'"
        educ.append(row[2])
        gender.append(row[5])
        status.append(row[3])
column_educ = weighting_educ(gender, educ)
column_marital = weighting_maritalstatus(gender, status)
print(column_educ, "\n", column_marital)

[1.3675893730874404, 0.5632350899031745, 1.1625413520124173, 1.3675893730874404, 0.23587982005915092, 2.7973391440904574, 1.3675893730874404, 2.7973391440904574, 0.4788344688743175, 2.149351705201895, 1.0062019662798887, 0.4788344688743175, 0.23587982005915092, 1.0062019662798887, 0.8378202110689285, 0.8378202110689285, 0.5632350899031745, 1.3675893730874404, 0.6412062818711671, 0.8666041367198846, 0.23587982005915092, 0.23587982005915092, 0.23587982005915092, 0.23587982005915092, 0.6412062818711671, 0.5632350899031745, 0.4788344688743175, 0.8378202110689285, 0.651998555703716, 2.7973391440904574, 0.651998555703716, 1.9491406536160796, 2.7973391440904574, 2.850473580190332, 0.8378202110689285, 2.850473580190332, 1.1625413520124173, 2.850473580190332, 2.149351705201895, 0.651998555703716, 0.651998555703716, 0.4788344688743175, 1.3675893730874404, 0.8666041367198846, 1.1625413520124173, 1.0062019662798887, 0.8378202110689285, 0.6412062818711671, 0.6412062818711671, 2.149351705201895, 0.8

In [13]:
def gemiddelde(col_A, col_B):
    return [(a+b)/2 for a,b in zip(col_A, col_B)]
gemiddelde(column_educ, column_marital)
# print(len(column_educ))
# print(len(column_marital))

[0.9646869212548588,
 0.6484726844671234,
 0.8921044497380468,
 1.0506498260592563,
 0.4662032110758,
 1.964709984417675,
 1.0506498260592563,
 1.7655247115607646,
 0.7603608208920043,
 1.3555680873120861,
 0.8513642841861688,
 0.6590512601970475,
 0.5375739357894642,
 0.9227350088998332,
 0.7671734065806888,
 0.9398536919893097,
 0.5924513186834255,
 1.0506498260592563,
 0.6508313781776286,
 0.7635303056019873,
 0.44816814727162047,
 0.44816814727162047,
 0.638883496484421,
 0.638883496484421,
 0.668866441981808,
 0.5625097796627259,
 0.6590512601970475,
 0.838544131294353,
 0.8469428643067035,
 1.7655247115607646,
 0.8469428643067035,
 1.540610739180486,
 1.964709984417675,
 1.7920919296107019,
 0.838544131294353,
 1.7920919296107019,
 0.9481258155217447,
 1.7920919296107019,
 3.0913047260806783,
 0.656227515093903,
 0.656227515093903,
 0.7603608208920043,
 0.9646869212548588,
 0.8529360941198311,
 1.1473110883786548,
 1.02404456959479,
 0.9398536919893097,
 0.668866441981808,
 0.841